# Primer Intento de clasificador



Como el titulo lo dice, primer intento de clasificador usando como referencia el laboratorio x .
Se considera para este caso el archivo movies_final_imdb el cual contiene:
    
    movieId+nota_imbd_aprox+duracion+categoria+actores+directores
    
    
Nota : Falta agregar los directores y quizas las fechas para ve como cambia. 

In [2]:
## Librerias Basicas:
import csv
import numpy as np
import pandas as pd

#Metricas y Estadisticas:
from statsmodels.stats.weightstats import ttest_ind
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn import cross_validation


# Clasificadores : Tree, SVM, RandomForest, Regression, Naive Bayes,KNN......
from sklearn.svm import SVC  
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB  
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression



Cargamos los datasets.

In [3]:
movies = pd.read_csv("movies_final.csv", header = 0)
score = pd.read_csv("imdb.csv", header = 0)

movies_header = list(movies.columns.values)
movies = movies.as_matrix()

score = np.ravel(score.as_matrix())

# Clasificadores y Validación.





Esta nueva seccion calcula el promedio del accuracy de varios tipos de clasificadores y despues corre un test estadistico para determinar si existe una diferencia significativa entre el clasificador que se está probando y los otros.

~Algo asi como un cross validation.


Definimos las funciones del clasificador:

In [3]:
#Variar el num_test modifica la probabilidad de estar mas cerca del promedio al hacer un t-test. 
#Creo que con 35 es suficiente para ese tipo de distribuciones.

def run_classifier(clf, X, y, num_tests=10):
    scores = []
    reports = []
    
    for i in range(num_tests):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
        clf.fit(X_train, y_train)
        scores.append(clf.score(X_test, y_test))

    return np.array(scores)

Clasificadores usados:

In [ ]:
c0 = ("Base", DummyClassifier(strategy='stratified'))
c1 = ("SVM", SVC(kernel='linear'))
c2 = ("DT", DecisionTreeClassifier())
c3 = ("NB", GaussianNB())
c4 = ("KNN", KNeighborsClassifier(n_neighbors=5))
c5 = ("RandomForest", RandomForestClassifier(n_estimators=10))
c6 = ("LogisticRegression",LogisticRegression())

classifiers = [c0, c1, c2, c3, c4,c5,c6]
result_list = []

Corremos las pruebas para cada test. 

In [ ]:
#Funcion Pesada, correr solo una vez. 

for name, clf in classifiers:
    accuracys = run_classifier(clf ,movies , score)
    result_list.append((name, accuracys))
    

In [ ]:
#Impresion de los resultados

print("+ indica diferencia significativa\n")

for name1, results1 in result_list:
    print("Comparando %s - Accuracy: %.2f" % (name1, results1.mean()))
    for name2, results2 in result_list:
        if name1 == name2:
            continue

        _, p_value, __ = ttest_ind(results1, results2)
        
        if p_value <= 0.05:
            sig = "+"
        else:
            sig = ""
        print("%s:\t%.2f %s" % (name2, results2.mean(), sig))
    print()

# Predicciones de algunas peliculas

Usando la siguiente funcion, se puede imprimir mas facilmente los resultados y luego calcular algunas metricas importantes. 

In [ ]:
for clf_name, clf in classifiers:
    X_train, X_test, y_train, y_test = train_test_split(movies, score, test_size =  0.30)
    clf.fit(X_train, y_train)
        

In [ ]:
sample_movies = [("Toy Story", movies[0], score[0]),("Avatar", movies[3111], score[3111]),
                 ("Titanic",movies[458],score[458]),("Transformers",movies[4057],score[4057]),
                 ("Never Say Never",movies[4189],score[4189])]
predicted_scores = []

# Toma un vector de tuplas nombre,clasificador y otro de nombre_pelicula,atributos,score_original y
# trata de predecir el score de esto para luego compararlos.
def predict_scores(clf, sample_movies):    
    for clf_name , clf in clf:
        for sample_name, sample_data, sample_score in sample_movies:
            predicted_scores.append((clf_name, sample_name,clf.predict(sample_data),sample_score))
    return predicted_scores

res = predict_scores(classifiers, sample_movies)

def predict_scores_toString(predicted_scores):
    string = "Clf\t\t\tNombrePelicula\t\t\tNotaPredicha\t\tNotaReal\n"
    for item in predicted_scores:
        for elements in item:
            string = str(string) + str(elements) + "\t\t\t"
        string = str(string) + "\n"
    return string
        
#Muchos Warnings :(        

res2 = predict_scores_toString(res)
print res2

# Busqueda del mejor train test

A traves de graficos, trataremos de buscar el mejor test-train para cada clasificador.

In [ ]:
def create_intervals(number):
    intervals = []
    for i in range(number):
        intervals.append(float(i)/number)
    
    return intervals[1:]

In [ ]:

# Funcion que determina la mejor proporcion del conjunto de entrenamiento 
# y retorna la lista mas el mejor clasificador generado.
def best_test_size(clf, X, y, intervals, test_iterations):
    scores_final = []
    scores = []
    best_clf = []
    best_accuracy = 0
    
    for i in intervals:
        for j in range(test_iterations):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = i)
            clf.fit(X_train, y_train)
            clf_score = clf.score(X_test, y_test)
            scores.append(clf_score)
            if (best_accuracy < clf_score):
                best_accuracy = clf_score
                best_clf = clf
            
            
        
        #Agrego el promedio de los resultados.
        scores_final.append(sum(scores) / float(len(scores)))
        
        #Vacio las listas.
        scores = []
            
    print scores_final  
    return (np.array(scores_final),best_clf)

In [ ]:
import matplotlib.pyplot as plt

intervalos1 = 10

test_size_results = []
intervals = create_intervals(intervalos1)

for clf_name, clf in classifiers:
    print "CLF Actual: " + clf_name
    test_results = best_test_size (clf, movies, score, 10,2) 
    #print test_results
    plt.plot(create_intervals(10),test_results)
    plt.ylabel('Accuracy de '+ clf_name)
    plt.show()
    test_size_results = []

In [ ]:
Conclusion. Aparentemente, SVM con Logit

## Mejor Parametro KNN (numero de vecinos)

In [ ]:
def best_parameter_KNN(X, y,parameters,tsize):
    scores = []
    param = []
    
    for i in range(parameters):
        param.append(i+1)
        for j in range(tsize):
            results = []
            clf = KNeighborsClassifier(n_neighbors=i+1)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)
            clf.fit(X_train, y_train)
            results.append(clf.score(X_test, y_test))
        
        scores.append(sum(results) / float(len(results)))
        results = []
    #Imprimo el grafico!
                      
    print 'Accuracy de KNN con ' + str(parameters) + ' parametros y ' + str(tsize) + ' iteraciones x parametro.'
    print param
    plt.plot(param,scores)
    plt.ylabel('Accuracy de KNN') 
    plt.xlabel('Numero de Parametros')
    plt.show()
    
    

best_parameter_KNN(movies,score,30,15)


## Mejor Parametro RandomForest

In [ ]:
def best_parameter_RF(X, y,parameters,tsize):
    scores = []
    param = []
    
    for i in range(parameters):
        param.append(i+1)
        for j in range(tsize):
            results = []
            clf = RandomForestClassifier(n_estimators=i+1)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)
            clf.fit(X_train, y_train)
            results.append(clf.score(X_test, y_test))
        
        scores.append(sum(results) / float(len(results)))
        results = []
    #Imprimo el grafico!
                      
    print 'Accuracy de KNN con ' + str(parameters) + ' parametros y ' + str(tsize) + ' iteraciones x parametro.'
    print param
    plt.plot(param,scores)
    plt.ylabel('Accuracy de KNN') 
    plt.xlabel('Numero de Parametros')
    plt.show()
    
    

best_parameter_RF(movies,score,30,15)

### Test 2

Para llegar a este test se hizo lo siguiente :

    1.Se determino en el test1 cuales eran los clasificadores que daban mejores resultados.

    2.Se determino (solo para 0.30 el tamaño del dataset) cuales eran los mejores parametros para random forest tree y KNN.

Aun falta :

Determinar los mejores resultados (usando algun algoritmo greedy) cual es el optimo de parametros para knn y random forest.



In [ ]:
# Test 2 . Parametros :

#Intervalos por probar
intervals2 = [0.1,0.2,0.3,0.4,0.5]
#Numero de iteraciones probadas.
iteraciones2 = 1


test_size_results = []


c00 = ("Base", DummyClassifier(strategy='stratified'))
c11 = ("SVM", SVC(kernel='linear'))
c22 = ("KNN", KNeighborsClassifier(n_neighbors=30))
c33 = ("LogisticRegression",LogisticRegression())

classifiers2 = [c00, c11, c22, c33]
best_classifiers = []

for clf_name, clf in classifiers2:
    test_results2 = best_test_size (clf, movies, score, intervals2,iteraciones2) 
    best_classifiers.append(test_results2[1])
    
    #print test_results
    plt.plot(intervals2,test_results2[0])
    plt.ylabel('Accuracy de '+ clf_name)
    plt.show()
    test_size_results2 = []
    

In [ ]:
# Imprimo los classification report de los mejores clasificadores que se obtuvieron. 
clf_names = ["Dummy:\n", "SVM:\n", "KNN:\n", "Logistic Regression:\n"]
from sklearn.multiclass import OneVsRestClassifier
i = 0
for clf in best_classifiers:
    print clf_names[i]
    score_pred = OneVsRestClassifier(clf.predict(movies))
    print(classification_report(score,score_pred))
    i+=1

Ideas :
    Clasificadores Pipeline : Peliculas (mala,buena) -> Peliculas (malas, medias, buenas) -> Peliculas(muy malas,malas,medias,buenas,muy buenas).

Que porcentaje de peliculas no tiene suficientes metadatos. 
    
    

# Clasificadores Pipeline

In [4]:
# Cargamos el archivo imbd_mayor3 que tiene 2 clases : 1 si la nota es mayor que 3 y 0 si no.

scores_greater3 = pd.read_csv("imdb_mayor3.csv", header = 0)
scores_greater3 = np.ravel(scores_greater3.as_matrix())

greater3_clf = LogisticRegression()
scores = cross_validation.cross_val_score( greater3_clf,movies , scores_greater3, cv=15)
print scores

X_train, X_test, y_train, y_test = train_test_split(movies, scores_greater3, test_size = 0.30)
greater3_clf.fit(X_train, y_train)

print(classification_report(scores_greater3,greater3_clf.predict(movies)))



IOError: File imdb_mayor3.csv does not exist

# Arbol de clasificadores 
    
    Tratamos de asignarle sentido a lo que estamos clasificando a través de varios clasificadores ordenados en un arbol.
   
Usaremos Regresion Logistica.   
   
Aun resta por ver Clasificador OneVSAll

In [5]:
#Funciones Genericas

#Carga un archivo con scores
def load_archive(name):
    archive = pd.read_csv(name, header = 0)
    archive = np.ravel(archive.as_matrix())
    return archive

#Carga un archivo con meta-datos
def load_text_archive(name):
    movies = pd.read_csv(name, header = 0)
    movies_header = list(movies.columns.values)
    movies = movies.as_matrix()
    return movies

#Entrena a los clasificadores usando los dataset correspondientes.
def train_trees(classifiersTree, movies, score_datasets):
    trained_classifiersTree = []
    for i in range(len(classifiersTree)):
        X_train, X_test, y_train, y_test = train_test_split(movies_datasets[i], scores_datasets[i], test_size = 0.5)
        classifiersTree[i].fit(X_train, y_train)
        trained_classifiersTree.append(classifiersTree[i])
    return trained_classifiersTree
    


## Arbol 1

In [11]:
#Definiciones Previas

#Clasificadores
clf_cmp4 = LogisticRegression() # 0 si es menor que 4, 1 de lo contrario.
clf_1to4 = LogisticRegression() # Clasifica en un rango de notas de 1 a 4.
clf_cmp7 = LogisticRegression() # 0 si es menor que 7, 1 de lo contrario
clf_5to7 = LogisticRegression() # Clasifica de 4 a 7. 
clf_8to10 = LogisticRegression() # Clasifica de 8 a 10.

#Archivos y Arrays.

movies_0to4 = load_text_archive("movies_0to4.csv")
movies_5to7 = load_text_archive("movies_5to7.csv")
movies_8to10 = load_text_archive("movies_8to10.csv")

scores_cmp4 = load_archive("scores_cmp4.csv")
scores_0to4 = load_archive("scores_0to4.csv")
scores_cmp7 = load_archive("scores_cmp7.csv")
scores_5to7 = load_archive("scores_5to7.csv")
scores_8to10 =load_archive("scores_8to10.csv")

#Arrays Listos
classifiersTree = [clf_cmp4,clf_1to4,clf_cmp7,clf_5to7,clf_8to10 ] 
movies_datasets = [movies, movies_0to4, movies,movies_5to7,movies_8to10]
scores_datasets = [scores_cmp4, scores_0to4, scores_cmp7 ,scores_5to7,scores_8to10]


In [7]:
#Funciones.
#("Toy Story", movies[0], score[0])
#Entrenamos el arbol.!




In [12]:
# Definimos el arbol con la siguiente funcion:
def predict_with_clfTree(classifiersTree,value_to_predict):
    
    #Clasifico si la nota es mayor o menor que 4.
    if (classifiersTree[0].predict(value_to_predict)):
        #Clasifico si la nota es mayor o menor que 7
        if (classifiersTree[2].predict(value_to_predict)):
            return classifiersTree[4].predict(value_to_predict)
            #Si es menor clasifico 
        else:
            return classifiersTree[3].predict(value_to_predict)
        
    #Si es menor que 4: 
    else:
        return classifiersTree[1].predict(value_to_predict)



In [13]:
#A ver que tal funciona!

#Creamos el arbol con los clasificadores.

classifiersTree = train_trees(classifiersTree,movies_datasets,scores_datasets)

scores_predicted = []
for i in range(len(movies)):
    scores_predicted.append(predict_with_clfTree(classifiersTree,movies[i].reshape(1, -1))[0])


In [10]:
#Imprimimos el reporte
print classification_report(score,scores_predicted)



             precision    recall  f1-score   support

          2       0.43      0.38      0.40        16
          3       0.42      0.12      0.19        40
          4       0.44      0.33      0.38       171
          5       0.59      0.34      0.43       478
          6       0.57      0.74      0.64      1487
          7       0.69      0.61      0.65      1497
          8       0.69      0.65      0.67       728
          9       0.24      0.60      0.34        35

avg / total       0.62      0.62      0.61      4452



## Arbol 2

In [46]:
#Definiciones Previas

#Clasificadores
clf_cmp5 = LogisticRegression() # 0 si es menor que 5, 1 de lo contrario.
clf_equal5 = LogisticRegression() # 0 si es menor que 5, 1 si es 5.
clf_1to4 = LogisticRegression() # Clasifica en un rango de notas de 1 a 4.
clf_cmp8 = LogisticRegression() # 0 si es menor que 8, 1 de lo contrario
clf_6to7 = LogisticRegression() # Clasifica de 6 a 7. 
clf_8to10 = LogisticRegression() # Clasifica de 8 a 10.

#Archivos y Arrays.

movies_0to4 = load_text_archive("movies_0to4.csv")
movies_0to5 = load_text_archive("movies_0to5.csv")
movies_6to7 = load_text_archive("movies_6to7.csv")
movies_8to10 = load_text_archive("movies_8to10.csv")
movies_cmp8 = load_text_archive("movies_cmp8.csv")

scores_cmp5 = load_archive("scores_cmp5.csv")
scores_0to4 = load_archive("scores_0to4.csv")
scores_cmp8 = load_archive("scores_cmp8.csv")
scores_6to7 = load_archive("scores_6to7.csv")
scores_8to10 =load_archive("scores_8to10.csv")

#Arrays Listos
classifiersTree = [clf_cmp5,clf_equal5,clf_1to4,clf_cmp8,clf_6to7,clf_8to10 ] 
movies_datasets = [movies,movies_0to5, movies_0to4,movies_cmp8,movies_6to7,movies_8to10]
scores_datasets = [scores_cmp5,scores_0to5, scores_0to4, scores_cmp8  ,scores_6to7,scores_8to10]


NameError: name 'scores_0to5' is not defined

In [ ]:
#Predice y evalua.
def predict_with_clfTree2(classifiersTree,value_to_predict):
    
    #Clasifico si la nota es mayor o menor que 5.
    if (classifiersTree[0].predict(value_to_predict)):
        #Clasifico si la nota es mayor o menor que 8:
        if (classifiersTree[3].predict(value_to_predict)):
            #Si es mayor, clasifico entre 8-9
            return classifiersTree[5].predict(value_to_predict)
            
        else:
            #Si es menor clasifico en el rango 6-8
            return classifiersTree[4].predict(value_to_predict)
        
    #Si es menor que 5: 
    else:
        if(classifiersTree[1].predict(value_to_predict)):
            return 5
        else:
            return classifiersTree[2].predict(value_to_predict)


In [ ]:
scores_predicted2 = []
for i in range(len(movies)):
    scores_predicted2.append(predict_with_clfTree(classifiersTree,movies[i])[0])

In [ ]:
print classification_report(score,scores_predicted2)